# Persistent homology of TPMS

In [1]:
import os
import re

import numpy as np
import matplotlib.pyplot as plt

import gudhi as gd

## Read data

In [2]:
SIZE = 10
NUMBER_OF_POINTS = 1000

Gyroid_path = os.getcwd() + '/data/Gyroid/'
Schwarz_path = os.getcwd() + '/data/Schwarz_p/'

def get_number_of_point_cloud(filename):
    return int(re.findall(r'\d+', filename)[0])
    
pointcloud_dict_Gyroid = {get_number_of_point_cloud(filename): 
                          np.genfromtxt(Gyroid_path+filename, delimiter=',') for filename in os.listdir(Gyroid_path)}
pointcloud_dict_Schwarz = {get_number_of_point_cloud(filename): 
                           np.genfromtxt(Schwarz_path+filename, delimiter=',') for filename in os.listdir(Schwarz_path)}

porosity_dict_Gyroid = {
    0:0.8067675507064093,
    1:0.8311745554478613,
    2:0.8597057075572289,
    3:0.7305331398219569,
    4:0.8879308997064143,
    5:0.8967083671120025,
    6:0.9958673101090609,
    7:0.9893618237568852,
    8:0.989704485737428,
    9:0.9926712846031019,
    10:0.9917984900228716,
    11:0.9848277993633552,
    12:0.9756546766585104,
    13:0.9749051830787189,
    14:0.9803575029108431,
    15:0.9754378092875391,
    16:0.9799372695204531,
    17:0.980479661178222,
    18:0.9956057454846639,
    19:0.996571275660434,
}

porosity_dict_Schwarz = {
    0:0.7966015689798916,
    1:0.7771466611653515,
    2:0.7596816566346682,
    3:0.8874772457654722,
    4:0.7303992228874115,
    5:0.7074796092705641,
    6:0.6811776087328972,
    7:0.6678446850601187,
    8:0.6894643371806497,
    9:0.6853880041298247,
    10:0.6901169368666168,
    11:0.6716186087703235,
    12:0.6641041950447361,
    13:0.6705497241717165,
    14:0.666063675835873,
    15:0.6733950600980956,
    16:0.6803108819780369,
    17:0.6711111347615506,
    18:0.686131063506277,
    19:0.686816953802124,
}

d_param_dict = {
    0:-0.9850857963678311,
    1:-0.8343366615126921,
    2:-0.602925211425586,
    3:-0.35709910913454423,
    4:-0.35088529180876293,
    5:-0.2623206879754544,
    6:-0.08615109294964762,
    7:0.18464827439927678,
    8:0.4294859077816455,
    9:0.44617777626215216,
    10:0.465394608817425,
    11:0.583935658785212,
    12:0.6425499439208111,
    13:0.7132291852184753,
    14:0.7213150183216164,
    15:0.7784261431370918,
    16:0.8677782656328097,
    17:0.873779300317322,
    18:0.953246239519624,
    19:0.9595609819109678,
}

## Plot functions

In [3]:
def plot_diagram_giotto(diagram, homology_dimensions=None, plotly_params=None):
    """Plot a single persistence diagram.

    Parameters
    ----------
    diagram : ndarray of shape (n_points, 3)
        The persistence diagram to plot, where the third dimension along axis 1
        contains homology dimensions, and the first two contain (birth, death)
        pairs to be used as coordinates in the two-dimensional plot.

    homology_dimensions : list of int or None, optional, default: ``None``
        Homology dimensions which will appear on the plot. If ``None``, all
        homology dimensions which appear in `diagram` will be plotted.

    plotly_params : dict or None, optional, default: ``None``
        Custom parameters to configure the plotly figure. Allowed keys are
        ``"traces"`` and ``"layout"``, and the corresponding values should be
        dictionaries containing keyword arguments as would be fed to the
        :meth:`update_traces` and :meth:`update_layout` methods of
        :class:`plotly.graph_objects.Figure`.

    Returns
    -------
    fig : :class:`plotly.graph_objects.Figure` object
        Figure representing the persistence diagram.

    """
    # TODO: increase the marker size
    if homology_dimensions is None:
        homology_dimensions = np.unique(diagram[:, 2])

    diagram = diagram[diagram[:, 0] != diagram[:, 1]]
    diagram_no_dims = diagram[:, :2]
    posinfinite_mask = np.isposinf(diagram_no_dims)
    neginfinite_mask = np.isneginf(diagram_no_dims)
    max_val = np.max(np.where(posinfinite_mask, -np.inf, diagram_no_dims))
    min_val = np.min(np.where(neginfinite_mask, np.inf, diagram_no_dims))
    parameter_range = max_val - min_val
    extra_space_factor = 0.02
    has_posinfinite_death = np.any(posinfinite_mask[:, 1])
    if has_posinfinite_death:
        posinfinity_val = max_val + 0.1 * parameter_range
        extra_space_factor += 0.1
    extra_space = extra_space_factor * parameter_range
    min_val_display = min_val - extra_space
    max_val_display = max_val + extra_space

    fig = gobj.Figure()
    fig.add_trace(gobj.Scatter(
        x=[min_val_display, max_val_display],
        y=[min_val_display, max_val_display],
        mode="lines",
        line={"dash": "dash", "width": 1, "color": "black"},
        showlegend=False,
        hoverinfo="none"
        ))

    for dim in homology_dimensions:
        name = f"H{int(dim)}" if dim != np.inf else "Any homology dimension"
        subdiagram = diagram[diagram[:, 2] == dim]
        unique, inverse, counts = np.unique(
            subdiagram, axis=0, return_inverse=True, return_counts=True
            )
        hovertext = [
            f"{tuple(unique[unique_row_index][:2])}" +
            (
                f", multiplicity: {counts[unique_row_index]}"
                if counts[unique_row_index] > 1 else ""
            )
            for unique_row_index in inverse
            ]
        y = subdiagram[:, 1]
        if has_posinfinite_death:
            y[np.isposinf(y)] = posinfinity_val
        fig.add_trace(gobj.Scatter(
            x=subdiagram[:, 0], y=y, mode="markers",
            hoverinfo="text", hovertext=hovertext, name=name
        ))

    fig.update_layout(
        width=500,
        height=500,
        xaxis1={
            "title": "Birth",
            "side": "bottom",
            "type": "linear",
            "range": [min_val_display, max_val_display],
            "autorange": False,
            "ticks": "outside",
            "showline": True,
            "zeroline": True,
            "linewidth": 1,
            "linecolor": "black",
            "mirror": False,
            "showexponent": "all",
            "exponentformat": "e"
            },
        yaxis1={
            "title": "Death",
            "side": "left",
            "type": "linear",
            "range": [min_val_display, max_val_display],
            "autorange": False, "scaleanchor": "x", "scaleratio": 1,
            "ticks": "outside",
            "showline": True,
            "zeroline": True,
            "linewidth": 1,
            "linecolor": "black",
            "mirror": False,
            "showexponent": "all",
            "exponentformat": "e"
            },
        plot_bgcolor="white"
        )

    # Add a horizontal dashed line for points with infinite death
    if has_posinfinite_death:
        fig.add_trace(gobj.Scatter(
            x=[min_val_display, max_val_display],
            y=[posinfinity_val, posinfinity_val],
            mode="lines",
            line={"dash": "dash", "width": 0.5, "color": "black"},
            showlegend=True,
            name=u"\u221E",
            hoverinfo="none"
        ))

    # Update traces and layout according to user input
    if plotly_params:
        fig.update_traces(plotly_params.get("traces", None))
        fig.update_layout(plotly_params.get("layout", None))

    return fig

In [3]:
def gudhi_dgm_to_giotto(dgm):
    """
    Transforms Gudhi diagram to Giotto-TDA diagram representation
    """
    size = len(dgm)
    giotto_dgm = np.zeros((size,3))
    homologies = np.array([cycle[0] for cycle in dgm])
    giotto_dgm[:,-1] = homologies
    birth_death_info = np.array([cycle[1] for cycle in dgm])
    giotto_dgm[:,:-1] = birth_death_info
    return giotto_dgm

## Persistence of Schwarz TPMS

In [4]:
dgms = {}
dgms_0 = {}
dgms_1 = {}
dgms_2 = {}
for i, pc in pointcloud_dict_Schwarz.items():
    print(f"{i}-th data is used for PH now")
    ac = gd.AlphaComplex(points=pc)
    stree = ac.create_simplex_tree()
    dgm = stree.persistence()
    dgms[i] = dgm
    dgms_0[i] = stree.persistence_intervals_in_dimension(0)
    dgms_1[i] = stree.persistence_intervals_in_dimension(1)
    dgms_2[i] = stree.persistence_intervals_in_dimension(2)

7-th data is used for PH now
6-th data is used for PH now
4-th data is used for PH now
5-th data is used for PH now
1-th data is used for PH now
0-th data is used for PH now
18-th data is used for PH now
2-th data is used for PH now
3-th data is used for PH now
19-th data is used for PH now
14-th data is used for PH now
15-th data is used for PH now
17-th data is used for PH now
16-th data is used for PH now
12-th data is used for PH now
8-th data is used for PH now
9-th data is used for PH now
13-th data is used for PH now
11-th data is used for PH now
10-th data is used for PH now


In [7]:
fig, axes = plt.subplots(figsize=(30,9), ncols=4, nrows=4, constrained_layout=True)
axes = axes.flatten()
for i in range(SIZE):
    gd.plot_persistence_diagram(dgms[i], axes=axes[i], fontsize=0)
    axes[i].set_title(f"Porosity={porosity_dict_Schwarz[i]:f}   d={d_param_dict[i]:f}")

fig.savefig('Schwarz_pd_per_porosity_size_10.png')
fig.suptitle('Schwarz TPMS with different $d$ values')
plt.show()

/Users/snopoff/miniconda3/envs/tpms/lib/python3.11/site-packages/gudhi/persistence_graphical_tools.py:85: UserWarning: There are 1437165 intervals given as input, whereas max_intervals is set to 1000000.
  warnings.warn(
/Users/snopoff/miniconda3/envs/tpms/lib/python3.11/site-packages/gudhi/persistence_graphical_tools.py:85: UserWarning: There are 1344325 intervals given as input, whereas max_intervals is set to 1000000.
  warnings.warn(
/Users/snopoff/miniconda3/envs/tpms/lib/python3.11/site-packages/gudhi/persistence_graphical_tools.py:85: UserWarning: There are 1351780 intervals given as input, whereas max_intervals is set to 1000000.
  warnings.warn(
/Users/snopoff/miniconda3/envs/tpms/lib/python3.11/site-packages/gudhi/persistence_graphical_tools.py:85: UserWarning: There are 1315659 intervals given as input, whereas max_intervals is set to 1000000.
  warnings.warn(
/Users/snopoff/miniconda3/envs/tpms/lib/python3.11/site-packages/gudhi/persistence_graphical_tools.py:85: UserWarnin

KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x137389940> (for post_execute):


KeyboardInterrupt: 

In [ ]:
fig, axes = plt.subplots(figsize=(30,9), ncols=5, nrows=4, constrained_layout=True)
axes = axes.flatten()
for i in range(SIZE):
    gd.plot_persistence_diagram(dgms_0[i], axes=axes[i], fontsize=0)
    axes[i].set_title(f"Porosity={porosity_dict_Schwarz[i]:f}   d={d_param_dict[i]:f}")

fig.savefig('Schwarz_pd_0_dim_per_porosity_size_10.png')
fig.suptitle('Schwarz TPMS with different $d$ values. Only 0-dimensional PDs')
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(30,9), ncols=5, nrows=4, constrained_layout=True)
axes = axes.flatten()
for i in range(SIZE):
    gd.plot_persistence_diagram(dgms_2[i], axes=axes[i], fontsize=0)
    axes[i].set_title(f"Porosity={porosity_dict_Schwarz[i]:f}   d={d_param_dict[i]:f}")

fig.savefig('Schwarz_pd_2_dim_per_porosity_size_10.png')
fig.suptitle('Schwarz TPMS with different $d$ values. Only 2-dimensional PDs')
plt.show()